In [ ]:
# L 4-30-25
# notebooks/classify_new_song.ipynb

In [8]:
# notebooks/3.1_Generate_Spectrograms.py
# notebooks/3.2_CNN_Spectrogram_Classifier.ipynb
# notebooks/classify_new_song.ipynb

# Modular inference pipeline setup for song classification using RF + CNN
# Spectrogram generation and CNN classifier are conditionally invoked via RUN_MODE

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import joblib
import runpy
from step_3_1_Gen_Spectro import generate_spectrograms_from_folder

RUN_MODE = "songs"

# Paths
SONG_DIR = "../data/Songs/"
SPECTRO_DIR = "../reports/4_Classify_New_Song/spectrograms/"
RF_OUTPUT_DIR = "../reports/4_Classify_New_Song/RF/"
CNN_OUTPUT_DIR = "../reports/4_Classify_New_Song/CNN/"
COMBINED_OUTPUT_PATH = "../reports/4_Classify_New_Song/all_model_predictions.csv"

# Ensure directories exist
os.makedirs(RF_OUTPUT_DIR, exist_ok=True)
os.makedirs(CNN_OUTPUT_DIR, exist_ok=True)
os.makedirs(SPECTRO_DIR, exist_ok=True)

# Step 1: Generate spectrograms for new songs
if RUN_MODE == "songs":
    generate_spectrograms_from_folder(
        song_dir=SONG_DIR,
        output_dir=SPECTRO_DIR,
        img_size=128
    )

# Step 2: Run CNN model to regenerate predictions
os.environ["RUN_MODE"] = "songs"
runpy.run_path("./step_3_2_CNN_Spectro.py", run_name="__main__")

# Step 3: Load Random Forest model
rf_model = joblib.load("../models/rf_model.pkl")

# Step 4: Helper for MFCC extraction


def extract_mfcc(path):
    y, sr = librosa.load(path, sr=None, duration=30)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    return np.concatenate([np.mean(mfcc, axis=1), np.std(mfcc, axis=1)])


# Step 5: Load CNN predictions from 3.2 output
cnn_df = pd.read_csv(os.path.join(CNN_OUTPUT_DIR, "cnn_song_predictions.csv"))

# Step 6: Inference loop
results = []
for genre_dir in os.listdir(SONG_DIR):
    genre_path = os.path.join(SONG_DIR, genre_dir)
    if not os.path.isdir(genre_path):
        continue

    for fname in os.listdir(genre_path):
        if not fname.endswith(".mp3"):
            continue

        base = os.path.splitext(fname)[0]
        fpath = os.path.join(genre_path, fname)
        spectro_path = os.path.join(SPECTRO_DIR, genre_dir, f"{base}.png")

        try:
            mfcc_vector = extract_mfcc(fpath).reshape(1, -1)
            rf_pred = rf_model.predict(mfcc_vector)[0]
            cnn_pred = cnn_df.loc[cnn_df['file'] == fname,
                                  'CNN'].values[0] if fname in cnn_df['file'].values else "N/A"

            results.append({
                "file": fname,
                "true_genre": genre_dir,
                "RF": rf_pred,
                "CNN": cnn_pred,
                "spectrogram_path": spectro_path
            })
        except Exception as e:
            print(f"Error processing {fname}: {e}")

# Step 7: Save combined predictions
df = pd.DataFrame(results)
df.to_csv(COMBINED_OUTPUT_PATH, index=False)

# Step 8: Evaluation (RF and CNN)
for model in ["RF", "CNN"]:
    if model not in df.columns:
        continue

    y_true = df["true_genre"]
    y_pred = df[model]
    labels_sorted = sorted(set(y_true) | set(y_pred))

    report = classification_report(
        y_true, y_pred, labels=labels_sorted, zero_division=0)
    with open(os.path.join(RF_OUTPUT_DIR if model == "RF" else CNN_OUTPUT_DIR,
                           f"{model.lower()}_classification_report.txt"), "w") as f:
        f.write(report)

    cm = confusion_matrix(y_true, y_pred, labels=labels_sorted)
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, cmap='Blues', interpolation='nearest')
    plt.title(f"Confusion Matrix - {model}")
    plt.colorbar()
    plt.xticks(np.arange(len(labels_sorted)),
               labels_sorted, rotation=45, ha='right')
    plt.yticks(np.arange(len(labels_sorted)), labels_sorted)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(os.path.join(RF_OUTPUT_DIR if model == "RF" else CNN_OUTPUT_DIR,
                             f"{model.lower()}_confusion_matrix.png"))
    plt.close()

    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, labels=labels_sorted, zero_division=0)
    metrics = {"Precision": precision, "Recall": recall, "F1-Score": f1}

    for metric_name, values in metrics.items():
        plt.figure(figsize=(10, 5))
        plt.bar(labels_sorted, values)
        plt.title(f"{metric_name} per Genre - {model}")
        plt.ylabel(metric_name)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        fname = f"{model.lower()}_{metric_name.lower().replace('-', '_')}_bar.png"
        plt.savefig(os.path.join(RF_OUTPUT_DIR if model ==
                    "RF" else CNN_OUTPUT_DIR, fname))
        plt.close()

[INFO] Generating spectrograms from folder: ../data/Songs/
Using device: cpu
Loading spectrograms from: ../spectrograms
Epoch 1/5, Loss: 1.8587
Epoch 2/5, Loss: 1.6297
Epoch 3/5, Loss: 1.5625
Epoch 4/5, Loss: 1.4711
Epoch 5/5, Loss: 1.3971
               precision    recall  f1-score   support

   Electronic       0.47      0.68      0.55       208
 Experimental       0.53      0.11      0.18       207
         Folk       0.46      0.70      0.55       199
      Hip-Hop       0.63      0.51      0.56       192
 Instrumental       0.44      0.60      0.51       202
International       0.47      0.54      0.50       215
          Pop       0.27      0.07      0.12       191
         Rock       0.57      0.68      0.62       185

     accuracy                           0.49      1599
    macro avg       0.48      0.49      0.45      1599
 weighted avg       0.48      0.49      0.45      1599



FileNotFoundError: [Errno 2] No such file or directory: '../reports/4_Classify_New_Song/CNN/cnn_song_predictions.csv'